In [1]:
!pip install -qU rich
!pip install -qU wandb weave
!pip install -qU llama-index
!pip install -qU llama-index-embeddings-huggingface llama-index-llms-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.1/328.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import rich
from google.colab import userdata

import wandb
import weave

from llama_index.core import (
    ServiceContext, StorageContext, load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

In [2]:
wandb.init(project="groq-rag", job_type="query")
weave.init(project_name="groq-rag")

config = wandb.config
config.embed_model_name = "BAAI/bge-small-en-v1.5"
config.groq_llm = "mixtral-8x7b-32768"
config.similarity_top_k = 10

artifact = wandb.use_artifact(
    "geekyrakshit/groq-rag/ncert-flamingoes-prose-embeddings:latest",
    type="vector_index"
)
artifact_dir = artifact.download()

wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


Logged in as Weights & Biases user: geekyrakshit.
View Weave data at https://wandb.ai/geekyrakshit/groq-rag/weave


wandb:   5 of 5 files downloaded.  


In [3]:
llm = Groq(model=config.groq_llm, api_key=userdata.get("GROQ_API_KEY"))
embed_model = HuggingFaceEmbedding(model_name=config.embed_model_name)
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, llm=llm
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<ipython-input-3-3745ef5094b9>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecate

In [4]:
storage_context = StorageContext.from_defaults(persist_dir=artifact_dir)
index = load_index_from_storage(
    storage_context, service_context=service_context
)

In [5]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=config.similarity_top_k,
    # node_postprocessors=[cohere_rerank],
)

In [6]:
query = """In the story \"The Last Lesson\",
what was the mood in the classroom when M. Hamel gave his last French lesson?
"""
response = query_engine.query(query).response

rich.print(response)

The mood in the classroom during M. Hamel's last French lesson was solemn and attentive. A sense of urgency and 
importance was attached to the lesson, as it was the final French lesson in that school. The students were quiet 
and focused, trying to absorb as much as they could from the lesson. M. Hamel himself was emotional and nostalgic, 
taking the time to appreciate the familiar sights of the classroom and the garden outside the window. The village 
people sitting quietly at the back benches added to the solemn atmosphere of the classroom. Overall, the mood was 
one of sadness, nostalgia, and a determination to make the most of the last French lesson. The new context does not
provide any additional information that would change the original answer.

In [7]:
table = wandb.Table(columns=["query", "response"], data=[[query, response]])
wandb.log({"response": table})

In [8]:
wandb.finish()